In [1]:
from models.SSGC import SSGC
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl
from time import perf_counter

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cpu'
activation = nn.ReLU()
epochs = 50
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [3]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [4]:
model = SSGC(in_feats, n_classes, K=2, alpha=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [5]:
def test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')
    print(f'Time: {perf_counter()-t:.2f}s')

In [7]:
train(model, g, features, labels, train_mask, val_mask, test_mask)
test(model, g, features, labels, test_mask)

Epoch 01, Loss: 1.1188, Train: 0.5333, Val: 0.4960, Test: 0.5380
Epoch 02, Loss: 1.0162, Train: 0.7000, Val: 0.5900, Test: 0.6140
Epoch 03, Loss: 0.8813, Train: 0.8167, Val: 0.6660, Test: 0.6690
Epoch 04, Loss: 0.7598, Train: 0.8000, Val: 0.6760, Test: 0.6910
Epoch 05, Loss: 0.7084, Train: 0.8500, Val: 0.6960, Test: 0.6960
Epoch 06, Loss: 0.6398, Train: 0.8833, Val: 0.7000, Test: 0.6920
Epoch 07, Loss: 0.5797, Train: 0.8833, Val: 0.6900, Test: 0.6830
Epoch 08, Loss: 0.5491, Train: 0.8667, Val: 0.6920, Test: 0.6780
Epoch 09, Loss: 0.5244, Train: 0.8833, Val: 0.6900, Test: 0.6870
Epoch 10, Loss: 0.4904, Train: 0.8833, Val: 0.7080, Test: 0.6920
Epoch 11, Loss: 0.4580, Train: 0.9167, Val: 0.7100, Test: 0.6960
Epoch 12, Loss: 0.4335, Train: 0.9167, Val: 0.7140, Test: 0.7080
Epoch 13, Loss: 0.4142, Train: 0.9167, Val: 0.7260, Test: 0.7200
Epoch 14, Loss: 0.3975, Train: 0.9167, Val: 0.7260, Test: 0.7240
Epoch 15, Loss: 0.3824, Train: 0.9167, Val: 0.7320, Test: 0.7270
Epoch 16, Loss: 0.3681, T

0.767